In [38]:
import pandas as pd
import requests
import finnhub
import time
from datetime import datetime
from datetime import date
from datetime import timedelta
pd.set_option('display.max_rows', None)
import pymongo

connstr = 'mongodb://localhost:27017'
client = pymongo.MongoClient(connstr)
db = client['crypto_dashboard']
user = db['user']
symbols = db['symbols']
app = db['app']
fh = finnhub.Client(api_key="c5tec52ad3ifck7dehd0")
dataLoc = '../Data/'
favList = user.find_one({})['favorites']
lastSyncDate = app.find_one({})['lastSyncDate']

In [26]:
def calculateSupertrend(data):
    data['prev-close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['prev-close'])
    data['low-pc'] = abs(data['low'] - data['prev-close'])
    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)
    data['tr'] = tr
    period = 10
    atr = data['tr'].rolling(period).mean()
    atr_multiplier = 3
    hl2 = (data['high'] + data['low']) / 2
    data['atr'] = atr
    data['upperband'] = hl2 + (atr_multiplier * data['atr'])
    data['lowerband'] = hl2 - (atr_multiplier * data['atr'])
    data['in_uptrend'] = True
    
    for current in range(1, len(data.index)):
        previous = current - 1
        if data.at[current, 'close'] > data.at[previous, 'upperband']:
            data.at[current, 'in_uptrend'] = True
        elif data.at[current, 'close'] < data.at[previous, 'lowerband']:
            data.at[current, 'in_uptrend'] = False
        else:
            data.at[current, 'in_uptrend'] = data.at[previous, 'in_uptrend']

            if data.at[current, 'in_uptrend'] and data.at[current, 'lowerband'] < data.at[previous, 'lowerband']:
                data.at[current, 'lowerband'] = data.at[previous, 'lowerband']

            if not data.at[current, 'in_uptrend'] and data.at[current, 'upperband'] > data.at[previous, 'upperband']:
                data.at[current, 'upperband'] = data.at[previous, 'upperband']
    return data

In [30]:
df = pd.read_csv('../Data/ADA_1H.csv')

In [31]:
df

,Unnamed: 0,date,dateDay,dateMonth,dateYear,dateHour,dateMinute,dateSecond,open,close,high,low,volume
0,0,2021-10-11 15:30:00,11,10,2021,15,30,0,2.225,2.205,2.232,2.203,3784542.5
1,1,2021-10-11 16:30:00,11,10,2021,16,30,0,2.206,2.208,2.220,2.198,6460507.1
2,2,2021-10-11 17:30:00,11,10,2021,17,30,0,2.208,2.209,2.216,2.199,3334273.6
3,3,2021-10-11 18:30:00,11,10,2021,18,30,0,2.209,2.216,2.220,2.201,4636627.4
4,4,2021-10-11 19:30:00,11,10,2021,19,30,0,2.215,2.208,2.217,2.198,6367886.9
5,5,2021-10-11 20:30:00,11,10,2021,20,30,0,2.208,2.202,2.213,2.199,4073770.5
6,6,2021-10-11 21:30:00,11,10,2021,21,30,0,2.202,2.192,2.204,2.189,4194118.3
7,7,2021-10-11 22:30:00,11,10,2021,22,30,0,2.191,2.186,2.198,2.179,3873149.6
8,8,2021-10-11 23:30:00,11,10,2021,23,30,0,2.185,2.179,2.188,2.171,4412398.7
9,9,2021-10-12 00:30:00,12,10,2021,0,30,0,2.179,2.155,2.192,2.153,7479939.0


In [39]:
def getChanges(df):
    stdf = calculateSupertrend(df)
    last = stdf.iloc[-1]['in_uptrend']
    pen = stdf.iloc[-2]['in_uptrend']
    return last, pen

In [41]:
print('Hourly Status:\n')
for symbol in favList:
    print(symbol)
    df = pd.read_csv(dataLoc+symbol+'_1H.csv')
    last, pen = getChanges(df)
    if last==pen:
        print('No change...')
    else:
        if last==True:
            print('In uptrend.. Buy..')
        else:
            print('In downtrend.. Sell..')

Hourly Status:

BTC
No change...
ETH
No change...
ADA
No change...
SOL
No change...
XRP
No change...
MATIC
No change...
BNB
No change...
LINK
No change...
LTC
No change...
EOS
No change...
BTT
No change...
TRX
No change...
VET
No change...
QNT
No change...
CAKE
No change...
HBAR
No change...
